In [57]:
import numpy as np
import math
import random 

import tensorflow as tf
from tensorflow import keras 


In [172]:
with open('datasets/jaychou_lyrics.txt') as fin:
    corpus_chars = fin.read()
    
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', '').replace(' ', '')
print("corpus_chars", len(corpus_chars))
# corpus_chars = corpus_chars[:20000]
# print("corpus_chars", len(corpus_chars))

idx2char = list(set(corpus_chars))
print("idx2char", len(idx2char))

char2idx = {
    ch: i
    for i, ch in enumerate(idx2char)
}
print("char2idx", len(char2idx))

vocab_size = len(char2idx)
print("vocab_size", vocab_size)

corpus_indices = [char2idx[ch] for ch in corpus_chars]
print("corpus_indices", len(corpus_indices))

corpus_chars 54473
idx2char 2581
char2idx 2581
vocab_size 2581
corpus_indices 54473


In [173]:
def data_iter_random(corpus_indices, batch_size, num_steps):
    # 在随机采样中，每个样本是原始序列上任意截取的一段序列。
    # 相邻的两个随机小批量在原始序列上的位置不一定相毗邻。
    # 因此，我们无法用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态。
    # 在训练模型时，每次随机采样前都需要重新初始化隐藏状态。
    corpus_size = len(corpus_indices)
    # 减1是因为输出的索引是相应输入的索引加1
    # An example contains num_steps elements of corpus
    num_examples = (corpus_size - 1)  // num_steps
    # A batch contains batch_size examples
    num_batches = num_examples // batch_size
    example_indices = list(range(num_examples))
    random.shuffle(example_indices)
    for batch_idx in range(num_batches):
        X = []
        Y = []
        for example_idx in example_indices[batch_idx * batch_size: (batch_idx+1) * batch_size]:
            X.append(corpus_indices[example_idx * num_steps: (example_idx+1) * num_steps])
            Y.append(corpus_indices[example_idx * num_steps + 1: (example_idx+1) * num_steps + 1])
        # output size : batch_size * num_steps
        yield np.array(X), np.array(Y)

        
def data_iter_consecutive(corpus_indices, batch_size, num_steps, ctx=None):
    # 可以令相邻的两个随机小批量在原始序列上的位置相毗邻。
    # 这时候，我们就可以用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态，
    # 从而使下一个小批量的输出也取决于当前小批量的输入，并如此循环下去。
    corpus_size = len(corpus_indices)
    # A batch contains batch_size examples
    num_steps_per_batch = corpus_size // batch_size  # ?
    indices = np.array(corpus_indices[: batch_size*num_steps_per_batch]).reshape([batch_size, num_steps_per_batch])
    num_batches = (num_steps_per_batch - 1) // num_steps
    for batch_idx in range(num_batches):
        X = indices[:, batch_idx * num_steps: (batch_idx+1) * num_steps]
        Y = indices[:, batch_idx * num_steps + 1: (batch_idx+1) * num_steps + 1]
        # output size : batch_size * num_steps
        yield X, Y

# my_seq = list(range(30))
# print("data_iter_random")
# for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=6):
#     print('X: ', X, '\nY:', Y, '\n')

# print("data_iter_consecutive")
# for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=6):
#     print('X: ', X, '\nY:', Y, '\n')


In [174]:

def get_params(num_inputs, num_hiddens, num_outputs):
    W_xh = tf.Variable(
        tf.random.normal(
            shape=[num_inputs, num_hiddens], 
            mean=0, stddev=0.01, 
            dtype=tf.float32
        )
    )
    W_hh = tf.Variable(
        tf.random.normal(
            shape=[num_hiddens, num_hiddens], 
            mean=0, stddev=0.01, 
            dtype=tf.float32
        )
    )
    b_h = tf.Variable(
        tf.zeros(num_hiddens), 
        dtype=tf.float32
    )
    
    W_hq = tf.Variable(
        tf.random.normal(
            shape=[num_hiddens, num_outputs], 
            mean=0, stddev=0.01, 
            dtype=tf.float32
        )
    )
    b_q = tf.Variable(
        tf.zeros(num_outputs), 
        dtype=tf.float32
    )
    return [W_xh, W_hh, b_h, W_hq, b_q]


def rnn(Xs, H, params):
    """
    :param Xs: num_steps * (batch_size, num_inputs)
    :param H: (batch_size, num_hiddens)
    :return 
        Ys: num_steps * (batch_size, num_outputs)
        H: (batch_size, num_hiddens)
    """
    W_xh, W_hh, b_h, W_hq, b_q = params
    Ys = []
    for X in Xs:  # X: (batch_size, vocab_size)
        H = tf.tanh(tf.matmul(X, W_xh) + tf.matmul(H, W_hh) + b_h)
        Y = tf.matmul(H, W_hq) + b_q  # (batch_size, num_outputs)
        Ys.append(Y)
    return Ys, H


def predict_rnn(prefix, num_chars, rnn, params, num_hiddens, vocab_size, idx2char, char2idx):
    state = tf.zeros([1, num_hiddens])
    output = [
        char2idx[prefix[0]]
    ]
    for t in range(num_chars + len(prefix) - 1):
        X = tf.one_hot([output[-1]], vocab_size)  # (1, vocab_size)
        Ys, state = rnn([X], state, params)
        Y = Ys[0]
        if t < len(prefix) - 1:
            pred = char2idx[prefix[t+1]]
        else:
            pred = int(np.array(tf.argmax(Y, axis=1)))
        output.append(pred)
    return ''.join([idx2char[i] for i in output])


def grad_clipping(grads, theta):
    norm = 0.0
    for i in range(len(grads)):
        norm += tf.math.reduce_sum(grads[i] ** 2)
    norm = np.sqrt(norm)
    if norm <= theta:
        return grads
    return [grad * theta / norm for grad in grads]


In [178]:
num_inputs = vocab_size
num_hiddens = 256
num_outputs = vocab_size

num_steps = 10
batch_size = 128
prefixes = ['分开', '不分开']
is_random_iter = True

optimizer = keras.optimizers.SGD(10)

data_iter_fn = data_iter_random if is_random_iter else data_iter_consecutive
params = get_params(num_inputs, num_hiddens, num_outputs)
for ep in range(200):
    # 如使用相邻采样，在epoch开始时初始化隐藏状态
    if not is_random_iter:
        state = tf.zeros([batch_size, num_hiddens])
    data_iter = data_iter_fn(corpus_indices, batch_size, num_steps)
    loss_sum = 0.0
    n = 0    
    for X, Y in data_iter:  # X, Y: batch_size * num_steps
        # 如使用随机采样，在每个小批量更新前初始化隐藏状态
        if is_random_iter:
            state = tf.zeros([batch_size, num_hiddens]) 
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(params)
            Xs = [  # num_steps * (batch_size, vocab_size)
                tf.one_hot(x, vocab_size, dtype=tf.float32)
                for x in X.T
            ]
            Ys, state = rnn(Xs, state, params)
            Y_pred = tf.concat(Ys, axis=0)  # (num_steps*batch_size, num_outputs)
            Y_true = tf.convert_to_tensor(
                Y.T.reshape((-1, )), 
                dtype=tf.float32
            )
            loss = tf.reduce_mean(
                tf.losses.sparse_categorical_crossentropy(Y_true, Y_pred)
            )
        grads = tape.gradient(loss, params)
        grads = grad_clipping(grads, 0.01)
        optimizer.apply_gradients(zip(grads, params))
        loss_sum += loss * len(Y_true)
        n += len(Y_true)
    if (ep + 1) % 20 == 0:
        """
        困惑度是对交叉熵损失函数做指数运算后得到的值。特别地，
        最佳情况下，模型总是把标签类别的概率预测为1，此时困惑度为1；
        最坏情况下，模型总是把标签类别的概率预测为0，此时困惑度为正无穷；
        基线情况下，模型总是预测所有类别的概率都相同，此时困惑度为类别个数。
        显然，任何一个有效模型的困惑度必须小于类别个数。在本例中，困惑度必须小于词典大小vocab_size。
        """
        print('epoch %d, perplexity %f' % (ep + 1, math.exp(loss_sum / n)))
        for prefix in prefixes:
            print(prefix)
            print(' -', predict_rnn(prefix, 30, rnn, params, num_hiddens, vocab_size, idx2char, char2idx))

epoch 20, perplexity 1551.870303
分开
 - 分开的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
不分开
 - 不分开的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
epoch 40, perplexity 655.144389
分开
 - 分开大大大大大大大大大大大大大大大大大大大大大大大大大大大大大大
不分开
 - 不分开大大大大大大大大大大大大大大大大大大大大大大大大大大大大大大
epoch 60, perplexity 335.174087
分开
 - 分开始烟一直一直一直一直一直一直一直一直一直一直一直一直一直一直
不分开
 - 不分开始烟一直一直一直一直一直一直一直一直一直一直一直一直一直一直
epoch 80, perplexity 252.423657
分开
 - 分开始MMMMMMMMMMMMMMMMMMMMMMMMMMMMM
不分开
 - 不分开始MMMMMMMMMMMMMMMMMMMMMMMMMMMMM
epoch 100, perplexity 444.741905
分开
 - 分开始终于她的爱情绪为什么会有一直一直一直一直一直一直一直一直一
不分开
 - 不分开始终于她的爱情绪为什么会有一直一直一直一直一直一直一直一直一
epoch 120, perplexity 1122.639064
分开
 - 分开始终每一种每天每天每天每天每天每天每天每天每天每天每天每天每
不分开
 - 不分开始终每一种每天每天每天每天每天每天每天每天每天每天每天每天每
epoch 140, perplexity 7741.999924
分开
 - 分开始移动的师读你的师读你的师读你的师读你的师读你的师读你的师读
不分开
 - 不分开始移动的师读你的师读你的师读你的师读你的师读你的师读你的师读
epoch 160, perplexity 2581.001324
分开
 - 分开伽照奶針陰率U尼b坟陰率U尼b坟陰率U尼b坟陰率U尼b坟陰率
不分开
 - 不分开伽照奶針陰率U尼b坟陰率U尼b坟陰率U尼b坟陰率U尼b坟陰率
epoch 180, perplexity 2581.001324
分开
 - 分开伽照奶針陰率U尼b坟陰率U尼b坟陰率U尼b坟陰率U尼b坟陰率
不分开
 - 不分开伽照奶針陰率U尼b坟陰率U